In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image

In [6]:
# --- Configuration ---
base_dir = "../../data/image_dataset"

# --- Script to find bad files ---
print("Scanning for corrupted or invalid images...")
for root, dirs, files in os.walk(base_dir):
    for filename in files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(root, filename)
            try:
                with Image.open(filepath) as img:
                    img.verify()
            except Exception as e:
                print(f"Problem file found: {filepath}")
                print(f"   Error: {e}\n")

print(" Scan complete. Please delete any problem files listed above.")

Scanning for corrupted or invalid images...
 Scan complete. Please delete any problem files listed above.


In [33]:
# --- Configuration ---
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')

img_height, img_width = 128, 128
batch_size = 32

# --- Data Augmentation & Loading ---
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# --- Get Class Labels ---
class_labels = sorted(list(train_generator.class_indices.keys()))
num_classes = len(class_labels)
print(f"Found {train_generator.samples} training images belonging to {num_classes} classes.")
print(f"Found {validation_generator.samples} validation images.")
print("Class Labels:", class_labels)

Found 6607 images belonging to 6 classes.
Found 320 images belonging to 6 classes.
Found 6607 training images belonging to 6 classes.
Found 320 validation images.
Class Labels: ['broken_streetlight', 'electric_pole_damage', 'flood_waterlogging', 'garbage', 'open_manhole', 'pothole']


In [8]:
input_shape = (img_height, img_width, 3)
inputs = Input(shape=input_shape)

# CNN Feature Extractor
x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)

# Prepare for RNN
reshaped = Reshape((x.shape[1], x.shape[2] * x.shape[3]))(x)

# RNN Classifier
x = LSTM(128, return_sequences=False)(reshaped)
x = Dropout(0.5)(x)

# Final Dense Layer
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 16, 2048)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 128)                 │       1,114,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,208,646 (4.61 MB)

 Trainable params: 1,208,646 (4.61 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# --- Training Parameters ---
epochs = 25 # Increase if needed, e.g., to 50

# --- Start Training ---
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    # Adjust steps if you have a very large dataset, otherwise it's fine
    steps_per_epoch=max(1, train_generator.samples // batch_size),
    validation_steps=max(1, validation_generator.samples // batch_size)
)

C:\Users\SWARNABHA\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
 19/206 ━━━━━━━━━━━━━━━━━━━━ 2:26 782ms/step - accuracy: 0.3313 - loss: 1.5594

C:\Users\SWARNABHA\AppData\Roaming\Python\Python311\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


206/206 ━━━━━━━━━━━━━━━━━━━━ 161s 757ms/step - accuracy: 0.5887 - loss: 1.0740 - val_accuracy: 0.8375 - val_loss: 0.6177
Epoch 2/25
  1/206 ━━━━━━━━━━━━━━━━━━━━ 1:14 366ms/step - accuracy: 0.7812 - loss: 0.7693

C:\Users\SWARNABHA\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


206/206 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.7812 - loss: 0.7693 - val_accuracy: 0.8625 - val_loss: 0.5547
Epoch 3/25
206/206 ━━━━━━━━━━━━━━━━━━━━ 144s 701ms/step - accuracy: 0.7316 - loss: 0.7539 - val_accuracy: 0.8531 - val_loss: 0.5787
Epoch 4/25
206/206 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8750 - loss: 0.4342 - val_accuracy: 0.8500 - val_loss: 0.5872
Epoch 5/25
206/206 ━━━━━━━━━━━━━━━━━━━━ 193s 675ms/step - accuracy: 0.7706 - loss: 0.6320 - val_accuracy: 0.8875 - val_loss: 0.4602
Epoch 6/25
206/206 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7500 - loss: 0.7912 - val_accuracy: 0.8781 - val_loss: 0.4611
Epoch 7/25
206/206 ━━━━━━━━━━━━━━━━━━━━ 137s 667ms/step - accuracy: 0.8014 - loss: 0.5618 - val_accuracy: 0.8469 - val_loss: 0.5924
Epoch 8/25
206/206 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.9688 - loss: 0.2062 - val_accuracy: 0.8500 - val_loss: 0.5921
Epoch 9/25
206/206 ━━━━━━━━━━━━━━━━━━━━ 142s 690ms/step - accuracy: 0.8198 - loss: 0.5102 - val_acc

In [10]:
# Save the entire model to a single HDF5 file.
model.save('../../models/civic_issue_image_model.h5')

print("Image model saved successfully as civic_issue_image_model.h5")

Image model saved successfully as civic_issue_image_model.h5
